## Describe the business problem
This notebook describes how to use CPLEX Modeling for Python to manage the production of pasta to meet demand with your resources.

The model aims at minimizing the production cost for a number of products while satisfying customer demand.

Each product can be produced either inside the company or outside, at a higher cost.
The inside production is constrained by the company's resources, while outside production is considered unlimited.
The model first declares the products and the resources.
The data consists of the description of the products (the demand, the inside and outside costs, and the resource consumption) and the capacity of the various resources. The variables for this problem are the inside and outside production for each product.

## Use decision optimization

#### Step 1: Model the data


In [38]:
# Products contain (name, demand, inside_cost, outside_cost)

products = [("kluski", 100, 0.6, 0.8),
            ("capellini", 200, 0.8, 0.9),
            ("fettucine", 300, 0.3, 0.4)]

# resources are a list of simple tuples (name, capacity)
resources = [("flour", 20),
             ("eggs", 40)]

# consumptions defined as (product_name, resource_named, consumed)
consumptions = {("kluski", "flour"): 0.5,
                ("kluski", "eggs"): 0.2,
                ("capellini", "flour"): 0.4,
                ("capellini", "eggs"): 0.4,
                ("fettucine", "flour"): 0.3,
                ("fettucine", "eggs"): 0.6}

#### Step 2: Prepare the data

The data used is very simple and is ready to use without any cleaning, massage, refactoring.

#### Step 3: Set up the prescriptive model

Set up the prescriptive model using the Mathematical Programming (docplex.mp) modeling package.

In [39]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Windows 64bit
* Python version 3.7.6, located at: C:\Users\User\anaconda3\python.exe
* docplex is present, version is 2.15.194
* CPLEX library is present, version is 12.10.0.0, located at: C:\Program Files\IBM\ILOG\CPLEX_Studio1210\cplex\python\3.7\x64_win64
* pandas is present, version is 1.0.1


#### Create the DOcplex model

The model contains all the business constraints and defines the objective.

We now use CPLEX Modeling for Python to build a Mixed Integer Programming (MIP) model for this problem.

In [40]:
from docplex.mp.model import Model
mdl = Model(name="pasta")

#### Define the decision variables

In [41]:
# This defines number of each products produced inside and outside the company
inside_vars = mdl.continuous_var_dict(products, name='inside')
outside_vars = mdl.continuous_var_dict(products, name='outside')

In [42]:
for p in products:
    print(inside_vars[p])

inside_kluski_100_0.6_0.8
inside_capellini_200_0.8_0.9
inside_fettucine_300_0.3_0.4


#### Express the business constraints

- Each product can be produced either inside the company or outside, at a higher cost.
- The inside production is constrained by the company's resources, while outside production is considered unlimited.

In [43]:
products

[('kluski', 100, 0.6, 0.8),
 ('capellini', 200, 0.8, 0.9),
 ('fettucine', 300, 0.3, 0.4)]

In [44]:
resources, consumptions

([('flour', 20), ('eggs', 40)],
 {('kluski', 'flour'): 0.5,
  ('kluski', 'eggs'): 0.2,
  ('capellini', 'flour'): 0.4,
  ('capellini', 'eggs'): 0.4,
  ('fettucine', 'flour'): 0.3,
  ('fettucine', 'eggs'): 0.6})

In [45]:
mdl.print_information()

Model: pasta
 - number of variables: 6
   - binary=0, integer=0, continuous=6
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: LP


In [46]:
# --- constraints ---
# demand satisfaction
# For each product, total production of inside and outside company >= demand
mdl.add_constraints((inside_vars[prod] + outside_vars[prod] >= prod[1], 'ct_demand_{}'.format(prod[0])) \
                    for prod in products)

# --- resource capacity ---
# For inside company, For each resource, total product consumptions must be less than capacity
# The operations give 2D-array for consumptions of the 2 resources in each product, p[flour, egg]
mdl.add_constraints((mdl.sum(inside_vars[p] * consumptions[p[0], res[0]] for p in products) <= res[1], 'ct_res_%{}'.format(res[0]))
                    for res in resources)

mdl.print_information()

Model: pasta
 - number of variables: 6
   - binary=0, integer=0, continuous=6
 - number of constraints: 5
   - linear=5
 - parameters: defaults
 - objective: none
 - problem type is: LP


#### Express the objective

Minimizing the production cost for a number of products while satisfying customer demand.

In [47]:
total_inside_cost = mdl.sum(inside_vars[p] * p[2] for p in products)
total_outside_cost = mdl.sum(outside_vars[p] * p[3] for p in products)

mdl.minimize(total_inside_cost + total_outside_cost)

#### Solve with Decision Optimization
Now solve the model, using Model.solve(). The following cell solves using your local CPLEX (if any, and provided you have added it to your PYTHONPATH variable).

In [48]:
mdl.solve()
mdl.print_information()

Model: pasta
 - number of variables: 6
   - binary=0, integer=0, continuous=6
 - number of constraints: 5
   - linear=5
 - parameters: defaults
 - objective: minimize
 - problem type is: LP


In [49]:
mdl.print_solution()

objective: 372.000
  inside_kluski_100_0.6_0.8=40.000
  outside_kluski_100_0.6_0.8=60.000
  outside_capellini_200_0.8_0.9=200.000
  outside_fettucine_300_0.3_0.4=300.000


In [50]:
mdl.objective_value

372.0

#### Step 4: Investigate the solution and then run an example analysis

print out solution in nice format

In [51]:
print('Objective value: {}'.format(mdl.objective_value))

print('Inside company variables:\n')
print('Total cost of production ==> {}'.format(total_inside_cost.solution_value))
for p in products:
    print('Total units of {} product ==> {}'.format(p[0], inside_vars[p].solution_value))

print('\nOutside company variables:\n')
print('Total cost of production ==> {}'.format(total_outside_cost.solution_value))
for p in products:
    print('Total units of {} product ==> {}'.format(p[0], outside_vars[p].solution_value))

Objective value: 372.0
Inside company variables:

Total cost of production ==> 24.0
Total units of kluski product ==> 40.0
Total units of capellini product ==> 0
Total units of fettucine product ==> 0

Outside company variables:

Total cost of production ==> 348.0
Total units of kluski product ==> 60.0
Total units of capellini product ==> 200.0
Total units of fettucine product ==> 300.0
